# To-Do List
Database

Front End

Image Quality - How to check

skimage.measure compare_SSIM

GT quality - Mean pixels strength, pixel mean value

Images w/ metadata
Metadata - Key Frames - Categories

Import batch of frames - Curate Frames - Check Ground Truth - Generate Metadata - Output
WandB
Flask Front End
Image Output

scalability

wandb - monitor quality

# Import FastAPI, FFmpeg, uvicorn, and JAAD

In [151]:
%%capture
!pip install fastapi ffmpeg uvicorn JAAD python-multipart tensorflow-gpu scikit-image imutils

from fastapi import FastAPI, File, UploadFile
import nest_asyncio, uvicorn

from io import BytesIO

# Import and Set Up WandB

## Import WandB Library

In [2]:
%%capture
!pip install wandb
import wandb

## Log In to WandB

In [3]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


True

# Set up ResNet50 Model

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]


2021-09-10 02:31:04.577938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-10 02:31:04.637083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-09-10 02:31:04.657723: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-09-10 02:31:04.658601: I tensorflow/core/platform/cpu_f

102981632/102967424 [==============================] - 1s 0us/step


# Set Up Google Cloud Parameters


## Import Google Cloud Library

In [7]:
from google.cloud import storage

## Set Up Google Cloud Project and Model Location

In [8]:
project = 'mlops-content1' # Cloud Project Name
location = 'james-mlops-capstone' # Model Storage Bucket
model_dir = 'model'

## Create Storage Bucket

In [100]:
storage_client = storage.Client.from_service_account_json('james-capstone-key.json')

bucket = storage_client.bucket(location)

# working_bucket = storage_client.bucket(location)

## Double Check Cloud Bucket

In [ ]:
%%capture
blobs = storage_client.list_blobs(location)
for blob in blobs:
    print(blob.name)

# WandB Functions

In [68]:
def init_wandb(project_name):
   global wandb_project
   wandb_project = str(project_name)
   wandb.init(project=wandb_project, sync_tensorboard=True)
   return True

# Set File Location

In [69]:
def set_folder_location(in_location):
   global location 
   location = str(in_location)
   return True

# Split Video to Frames and Upload

## Download Video to Local Instance

In [153]:
def download_video(video_name):
    blob = bucket.blob(video_name)
    blob.download_to_filename(video_name)
    
download_video("video_0001.mp4")

## Break down video to frames

In [245]:
def split_video_frames(video_name):
    folder = video_name[:-4]
    !mkdir $folder
    !ffmpeg -i $video_name $folder/frame%04d.png

split_video_frames("video_0001.mp4")

mkdir: cannot create directory ‘video_0001’: File exists
ffmpeg version 9c33b2f Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5. 

## Upload Video Frames

In [246]:
import glob

def upload_frames_from_folder(folder_name):
    folder_name = folder_name + "/*.png"
    print("Uploading Frames")
    for filename in glob.iglob(folder_name):
        print(filename, end="\r")
        blob = bucket.blob(filename)
        blob.upload_from_filename(filename)
    print("Done Uploading               ", end="\r")
    
upload_frames_from_folder("video_0001")

Uploading Frames


## SSIM Compare Video Frames for Novel Frames

## Import Libraries

In [247]:
# import the necessary packages
from skimage.metrics import structural_similarity as compare_ssim
import cv2

## Remove Blurry Images from Set

### Calculate Blurriness using Laplacian

In [248]:
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

### Remove Blurry Images

In [249]:
def remove_blurry_images(folder_name):
    files=sorted(os.listdir(str(folder_name)))
    files=files[1:]
    
    for file in files:
        print(file, end="\r")
        img=cv2.imread(folder_name+'/'+file)
        img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurriness=variance_of_laplacian(img_gray)
        if blurriness>100:
            os.remove(folder_name+'/'+file)
            print("Removed: " + file + " - Laplace Variance Blurriness: " + str(blurriness))
    print("Done Checking Frames")

remove_blurry_images("video_0001")

Done Checking Frames


# Deduplicate Similar Frames

## Calculate Similarity Between Images

In [250]:
def compare_images(image1, image2):
    image_gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image_gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    diff, _ = compare_ssim(image_gray1, image_gray2, full=True)
    return diff

## Remove Duplicates

In [256]:
def remove_duplicates(folder_name):
    files=sorted(os.listdir(str(folder_name)))
    files=files[1:]
    for i in range(len(files)-1):
        #print(files[i])#, end="\r")
        print(files[i] + "             ", end="\r")
        image1 = cv2.imread(folder_name+'/'+files[i])
        image2 = cv2.imread(folder_name+'/'+files[i+1])
        diff = compare_images(image1, image2)
        if diff > 0.99:
            print("Deleting " + files[i])
            os.remove(folder_name+'/'+files[i])
    print("Done Checking Frames           ")
    print()

remove_duplicates("video_0001")

Deleting frame0159.png    
Deleting frame0160.png    
Deleting frame0444.png    
Deleting frame0469.png    
Deleting frame0473.png    
Deleting frame0477.png    
Deleting frame0481.png    
Deleting frame0493.png    
Deleting frame0505.png    
Deleting frame0506.png    
Deleting frame0507.png    
Deleting frame0508.png    
Deleting frame0509.png    
Deleting frame0513.png    
Deleting frame0518.png    
Deleting frame0519.png    
Deleting frame0520.png    
Deleting frame0521.png    
Deleting frame0522.png    
Deleting frame0525.png    
Deleting frame0526.png    
Deleting frame0527.png    
Deleting frame0532.png    
Done Checking Frames           



# Import Labels

In [ ]:
!pip3 install pickle5
import pickle5 as pickle
import pandas as pd

with open('jaad_database.pkl', 'rb') as pickle_file:
    pickle_data = pickle.load(pickle_file)

labels = pd.DataFrame(data=pickle_data)
labels['video_0001']

# FastAPI Deployment

In [ ]:
app = FastAPI()

@app.on_event("startup")
def start_wandb(string_input="4B June 2021 Capstone"):
   init_wandb(project_name=string_input)
   return {'message': ('Weights and Balances Started as project: ' + wandb_project)}

@app.get('/')
def index():
    return {'message': 'This is the homepage of the model, add \'/docs\' to the end of the URL to access FastAPI to make predictions with the model'}

@app.get('/reload_model')
async def reload_model():
   global model
   model = model = tf.keras.models.load_model(BUCKET + f'/{model_dir}')
   return {'message': 'Model on GCP reloaded at ' + BUCKET + '/' + model_dir}

@app.get('/change_model')
async def change_model(string_input):
   global model_dir
   model_dir = str(string_input)
   model = tf.keras.models.load_model(BUCKET + f'/{model_dir}')
   return {'message': 'Model on GCP loaded from ' + BUCKET + '/' + model_dir, 'warning': 'This function doesn\'t currently check for file existence at location'}

@app.post('/predict_single')
async def predict_api(file: UploadFile = File(...)):
    extension = file.filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not extension:
        return "Image must be jpg or png format!"
    image = read_imagefile(await file.read())
    prediction = run_predict_single(image)
    prediction = str(prediction)
    print(prediction)
    return prediction

# Run Deployment

In [ ]:
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [64]
INFO:     Waiting for application startup.
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/starlette/routing.py", line 540, in lifespan
    async for item in self.lifespan_context(app):
  File "/usr/local/lib/python3.7/dist-packages/starlette/routing.py", line 481, in default_lifespan
    await self.startup()
  File "/usr/local/lib/python3.7/dist-packages/starlette/routing.py", line 518, in startup
    handler()
  File "<ipython-input-9-fa9cff6b7f9b>", line 5, in start_wandb
    wandb.init(project="mlops-midterm", sync_tensorboard=True)
NameError: name 'wandb' is not defined

ERROR:    Application startup failed. Exiting.
